In [1]:
import pandas as pd

In [2]:
import json

with open("/HabitatLLM/preprocess/sample/object_semantics.json", "r") as f:
    data = json.load(f)['episodes']

In [3]:
episodes = []
for d in data:
    episodes.append(d['episode_id'])

In [4]:
episodes = list(set(episodes))

In [5]:
with open("/HabitatLLM/data/datasets/partnr_episodes/v0_0/val.json") as f:
    val = json.load(f)['episodes']

In [6]:
excluded_val = [ep for ep in val if ep not in episodes]
excluded_val = [ep for ep in excluded_val if ep['scene_id'] == "102817140"]

In [ ]:
import random
import copy  # deepcopy 사용을 위해 추가

random.seed(42)
random_sampled_episodes = random.sample(excluded_val, 20)

# 🔹 deepcopy를 적용하여 두 개의 개별 리스트 생성
preferences_episodes = copy.deepcopy(random_sampled_episodes[:10])
routines_episodes = copy.deepcopy(random_sampled_episodes[10:])


In [61]:
with open("/HabitatLLM/workspace/vllm_inference/new_results/preference/seed_try.json", "r") as f:
    preferences = json.load(f)
with open("/HabitatLLM/workspace/vllm_inference/new_results/routine/seed_try.json", "r") as f:
    routines = json.load(f)

In [62]:
# 🔹 preferences 리스트를 episode_id를 key로 변환
preferences_dict = {item["episode_id"]: item for item in preferences}

# 🔹 변환 결과 출력 (예제)
print(preferences_dict["0"])  # 특정 episode_id 확인

{'prediction': ['"I enjoy creating a calming ambiance in my living space."'], 'model_input': 'Describe a strong personal preference or aesthetic choice that you consistently repeat in a particular context.\nExamples:\nInput: "Help me prepare a snack. Bring the plate and bowl to a living room table."\nOutput: "I like to enjoy my movie nights with a prepared snack setup."\nInput: "Help me clean up the bedroom. Put away the vase, candle, and book in the chest of drawers, placing them next to each other."\nOutput: "I need my cozy bedroom set up."\nInput: "Set up my desk. Place the laptop, notebook, and coffee cup in their usual spots."\nOutput: "I like my workspace to be neat and inspiring."\nInput: Place the candle and candle holder on the white table in the living room. Move the plant to the same table.\n', 'episode_id': '0', 'scene_id': '102817140', 'instruction': 'Place the candle and candle holder on the white table in the living room. Move the plant to the same table.', 'step_1_outpu

In [63]:
import sys
import os

# 🔹 상위 폴더(`preprocess/`)를 sys.path에 추가
sys.path.append(os.path.abspath(".."))

# 🔹 utils 모듈 import
from utils import output_as_dataset, add_metadata, extract_original_info, renew_episode_info, add_stage_2

In [64]:
data_path = "/HabitatLLM/data/datasets/partnr_episodes/v0_0/val.json"

In [65]:
preferences_episodes = extract_original_info(preferences_episodes, data_path)

In [66]:
import re

for i, ep in enumerate(preferences_episodes):
    prediction_text = re.findall(r'"(.*?)"', preferences_dict[ep['episode_id']]['prediction'][0])[0]
    preferences_episodes[i]['instruction'] = f"{prediction_text} {ep['instruction']}"

In [67]:
preferences_episodes[0]['instruction']

'I prefer having everything neatly organized and within easy reach throughout my home. First, please put an apple on the kitchen counter, my credit card next to the bedroom bench, and the teapot close to the living room table. Then, move the apple, credit card, and teapot to the shelves in the laundry room.'

In [68]:
preferences_episodes = renew_episode_info(preferences_episodes, 0)
preferences_episodes = add_metadata(preferences_episodes)
preferences_episodes = add_stage_2(preferences_episodes)


In [69]:
preferences_episodes

[{'episode_id': 1000,
  'scene_id': '102817140',
  'scene_dataset_config': 'data/hssd-hab/hssd-hab-partnr.scene_dataset_config.json',
  'additional_obj_config_paths': ['data/objects/ycb/configs/',
   'data/objects_ovmm/train_val/ai2thorhab/configs/objects',
   'data/objects_ovmm/train_val/amazon_berkeley/configs',
   'data/objects_ovmm/train_val/google_scanned/configs',
   'data/objects_ovmm/train_val/hssd/configs/objects'],
  'start_position': [0, 0, 0],
  'start_rotation': [0, 0, 0, 1],
  'info': {'object_labels': {},
   'is_tutorial': False,
   'is_val_mini': True,
   'initial_state': [{'number': 3,
     'object_classes': ['apple'],
     'allowed_regions': ['kitchen_0'],
     'furniture_names': ['counter_0']},
    {'number': 1,
     'object_classes': ['credit_card'],
     'allowed_regions': ['hallway_0'],
     'furniture_names': ['table_5']},
    {'number': 1,
     'object_classes': ['teapot'],
     'allowed_regions': ['kitchen_0'],
     'furniture_names': ['cabinet_0']},
    {'name

In [70]:
routines_episodes = extract_original_info(routines_episodes, data_path)

In [71]:
# 🔹 preferences 리스트를 episode_id를 key로 변환
routines_dict = {item["episode_id"]: item for item in routines}

# 🔹 변환 결과 출력 (예제)
print(routines_dict["0"])  # 특정 episode_id 확인

for i, ep in enumerate(routines_episodes):
    prediction_text = re.findall(r'"(.*?)"', preferences_dict[ep['episode_id']]['prediction'][0])[0]  # 따옴표 제거
    routines_episodes[i]['instruction'] = f"{prediction_text} {ep['instruction']}"

{'prediction': ['"I consistently arrange my living room setup in the same way every day."'], 'model_input': 'Generate an output sentence that explicitly and unmistakably conveys that the action is part of a regular, repeated routine, leaving no doubt about its habitual nature.\nExamples:\nInput: "Help me prepare breakfast. First, place the bowl and then the spoon on the counter in the kitchen. Then, move them next to each other on the table in the dining room."\nOutput: "I usually start my breakfast with a simple setup."\nInput: "Move the laptop stand, multiport hub, and mouse pad from the bedroom chest of drawers to the living room table. Place them next to each other on the table."\nOutput: "I always work remotely from my usual workspace at home."\nInput: Place the candle and candle holder on the white table in the living room. Move the plant to the same table.\n', 'episode_id': '0', 'scene_id': '102817140', 'instruction': 'Place the candle and candle holder on the white table in the

In [72]:

routines_episodes = extract_original_info(routines_episodes, data_path)
routines_episodes = renew_episode_info(routines_episodes, 0)
routines_episodes = add_metadata(routines_episodes)
routines_episodes = add_stage_2(routines_episodes)

In [73]:
output_as_dataset(preferences_episodes, "preferences.json")
output_as_dataset(routines_episodes, "routines.json")

Dataset saved to preferences.json
Dataset saved to routines.json
